In [10]:
import numpy as np
import matplotlib as plt
import pandas as pd
import yfinance as yf
import importlib
import dataReading 


## Lab 1 Enterprise Risk Management

## 1. Daily volatility forecast using MA and EWMA

Perform a daily volatility forecast using MA and EWMA with a 60 day estimation window, over the time period
2020-01-01 -> 2023-12-31 for the Nasdaq 100 index (^NDX). Plot the daily forecasts in the same plot. Plot also
the log-returns in a separate plot below.

*Discussion: What is the main reason for the differences in these two forecasts.*

In [16]:
# How to load all functions from a file into the notebook
importlib.reload(dataReading) # Needed so that you dont need to reload kernel when changing function
from dataReading import *

In [19]:
### To download data and store it as a csv file in the folder DataFileFolder. The filename is price_<TICKER>
# ticker = '^NDX'
# startDate = '2020-01-01'
# endDate = '2023-12-31'
# stockData = readDataFromYahoo(ticker, startDate, endDate)

# fileName = saveDataToCsv(stockData = stockData, stockName = ticker)

In [20]:
# To download data from file
stockDataFromFile = readDataFromFile('price_^NDX.csv')


## 2. Daily volatility forecast using GARCH

Fit a GARCH(1,1)-model using Nasdaq return data for the time period 2010-01-01 -> 2019-12-31 and then
perform a daily volatility forecast as in 1, with forecasting period: 2020-01-01 -> 2023-12-31. Plot your daily
forecast in the same plot as in 1.

*Discussion: What is the main reason for the difference between the GARCH and the EWMA forecast.*

## 3. GARCH model diagnostics

Analyze the GARCH-model and perform model diagnostics:
- a) What are the GARCH-parameters and what is the unconditional volatility?
- b) Plot the ACF (for lags 1 to 200) for the returns and the squared returns (use a stem-plot).
- c) Plot the ACF (for lags 1 to 200) for the residuals and the squared residuals (use a stem-plot).
- d) Plot a histogram (using 100 bins) of the residuals together with a normal pdf (same mean and std).
- e) QQ-plot the residuals against normal- and student t quantiles. What t-distribution provides the best fit
to the left tail?

*Discussion: What does b) and c) tell you? What does d), e) and f) tell you?*

## 4. VaR forecasts using historical simulation

Use backtesting to compare daily VaR98%-forecasts for the period 2020-01-01 -> 2023-12-31 using two versions
of basic historical simulation with rolling estimation windows of length 500 and 1000 days respectively.
- a) Plot the VaR-forecasts in the same plot as the (negative) returns
- b) Calculate the violation ratios for the different methods and give the number of violations for each
method in the report

*Discussion: What are the pros and cons of using these methods?*

## 5. VaR forecasts using GARCH and FHS

Repeat the backtest procedure in 4 but now based on the GARCH-model from 2. The two models to backtest
are:
- GARCH with the assumption of normally distributed conditional returns
- One-period filtered historical simulation (FHS)

You do not need to recalibrate the GARCH-model during the backtest period.
- a) Plot the VaR-forecasts in the same plot as in 4
- b) Calculate the violation ratios

*Discussion: What are the pros and cons of using these methods?*